In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"  # specify which GPU(s) to be used

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"  # specify which GPU(s) to be usedimport tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import random
import pandas as pd
import math
import tensorflow as tf

In [3]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [32]:
MAX_HEIGHT = 1072
MAX_WIDTH = 1912
shrink = 8
HEIGHT = MAX_HEIGHT // shrink
WIDTH = MAX_WIDTH // shrink
batch_size = 64
print(HEIGHT, WIDTH)

134 239


In [ ]:
tensor = tf.convert_to_tensor([1, 2, 3, 4], dtype=tf.float32)
tensor / 5

In [30]:
def get_classes_info(root_dir: str, height, width):
    labels = set()
    counter = 0
    classes_num = {}
    with open(os.path.join(root_dir, 'localization.txt'), 'r') as file:
        for line in file:
            line = line.strip()
            character = line.split(',')[-1]
            if character not in labels:
                labels.add(character)
                classes_num[character] = 0
            classes_num[character] += 1
            counter += 1
    labels = sorted(list(labels))
    return labels, classes_num, counter

def convertion(labels: list):
    convert_to_categorical = {character: ind for ind, character in enumerate(labels)}
    return convert_to_categorical

def preprocess_image(image, coords):
    image = tf.image.decode_jpeg(image, channels=3)
    height, width, channels = image.shape
    print(image.shape)
    x_1, y_1, x_2, y_2 = coords
    x_1, x_2 = x_1 * WIDTH / width, x_2 * WIDTH / width
    y_1, y_2 = y_1 * HEIGHT / height, y_2 * HEIGHT / height
    coords = tf.convert_to_tensor([x_1, y_1, x_2, y_2])
    print(coords)
    coords = tf.math.round(coords)
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image /= 255.0
    return image, coords


def load_and_preprocess(image_path, coords):
    image = tf.io.read_file(image_path)
    return preprocess_image(image, coords)

def parse_input(root_dir: str, height, width):
    images = []
    answers_coord = []
    answers_class = []
    file_path = os.path.join(root_dir, 'try.txt')
    print(file_path)
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip().split(',')
            x_1, y_1, x_2, y_2 = map(int, line[1:5])
            x_1, x_2 = min(x_1, x_2), max(x_1, x_2)
            y_1, y_2 = min(y_1, y_2), max(y_1, y_2)
            image_path = os.path.join(root_dir, line[0])
            coords = (x_1, y_1, x_2, y_2)
            print(coords)
            image, coords = load_and_preprocess(image_path, coords)
            print(coords)
            images.append(image)
            answers_coord.append(coords)
            answers_class.append(tf.keras.utils.to_categorical(convert_to_categorical[line[5]], len(labels)))
    return image_path, answers_coord, answers_class

In [31]:
labels, classes_num, total = get_classes_info('the-simpsons-characters-dataset', 0, 0)
# print(labels)
for character, cnt in sorted(classes_num.items(), key=lambda x: -x[1]):
    print(character, ':', cnt)
print('\n', total)
print(len(labels))
convert_to_categorical = convertion(labels)
image_path, answres_coord, answers_class = parse_input('the-simpsons-characters-dataset', 0, 0)
print(image_path)

charles_montgomery_burns : 650
homer_simpson : 612
abraham_grampa_simpson : 595
ned_flanders : 595
lisa_simpson : 562
marge_simpson : 557
bart_simpson : 554
principal_skinner : 506
krusty_the_clown : 226
nelson_muntz : 219
moe_szyslak : 215
kent_brockman : 213
edna_krabappel : 212
milhouse_van_houten : 210
chief_wiggum : 209
comic_book_guy : 208
apu_nahasapeemapetilon : 206
sideshow_bob : 171

 6720
18
the-simpsons-characters-dataset/try.txt
(80, 31, 337, 354)
(368, 496, 3)
tf.Tensor([ 38.548386  11.288043 162.38509  128.90218 ], shape=(4,), dtype=float32)
tf.Tensor([ 39.  11. 162. 129.], shape=(4,), dtype=float32)
the-simpsons-characters-dataset/simpsons_dataset/abraham_grampa_simpson/pic_0001.jpg
